In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
dataset_path = "dataset.csv"
df = pd.read_csv(dataset_path)

In [ ]:
df.head(50)

In [6]:
# Remove negative values from the dataset
for column in df.select_dtypes(include=['int64', 'float64']).columns:
    df = df[df[column] >= 0]

print('Negative values removed from the dataset.')

Negative values removed from the dataset.


In [4]:
# scaler = MinMaxScaler()
# scaled_features = scaler.fit_transform(df[['month', 'day', 'time', 'humidity', 'tempC']])
# print(scaled_features)

In [7]:
scaled_features = pd.DataFrame()
scaled_features['month'] = df['month'] / 12
scaled_features['day'] = df['day'] / 31
scaled_features['time'] = df['time'] / 24
scaled_features['humidity'] = df['humidity'] / 100
scaled_features['tempC'] = df['tempC'] / 50
print(scaled_features.head())

      month       day      time  humidity  tempC
0  0.083333  0.032258  0.000000      0.74   0.36
1  0.083333  0.032258  0.041667      0.78   0.36
2  0.083333  0.032258  0.083333      0.82   0.34
3  0.083333  0.032258  0.125000      0.85   0.34
4  0.083333  0.032258  0.166667      0.83   0.34


In [8]:
X = scaled_features
y = df['fan_speed'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [9]:
model = tf.keras.Sequential([

    tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2), input_shape=(5,)),
    tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
    tf.keras.layers.Dense(6, activation='softmax')  # Softmax activation for multi-class classification
])

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=90, validation_split=0.2)



Epoch 1/100


63/63 [==============================] - 2s 10ms/step - loss: 1.4926 - accuracy: 0.3909 - val_loss: 1.3448 - val_accuracy: 0.4196
Epoch 2/100
63/63 [==============================] - 0s 5ms/step - loss: 1.2631 - accuracy: 0.4414 - val_loss: 1.1522 - val_accuracy: 0.4893
Epoch 3/100
63/63 [==============================] - 0s 5ms/step - loss: 1.1128 - accuracy: 0.5079 - val_loss: 1.0398 - val_accuracy: 0.6003
Epoch 4/100
63/63 [==============================] - 0s 4ms/step - loss: 1.0061 - accuracy: 0.5684 - val_loss: 0.9434 - val_accuracy: 0.5960
Epoch 5/100
63/63 [==============================] - 0s 5ms/step - loss: 0.9308 - accuracy: 0.5969 - val_loss: 0.8849 - val_accuracy: 0.6565
Epoch 6/100
63/63 [==============================] - 0s 4ms/step - loss: 0.8714 - accuracy: 0.6257 - val_loss: 0.8419 - val_accuracy: 0.6366
Epoch 7/100
63/63 [==============================] - 0s 4ms/step - loss: 0.8127 - accuracy: 0.6534 - val_loss: 0.7646 - val_accuracy: 0.6906
Epoch 8/1

In [11]:
# Convert true classes to one-hot encoding
true_classes_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=6)
predicted_classes = model.predict(X_test)
# print(predicted_classes)
# Calculate accuracy
accuracy = np.mean(np.argmax(predicted_classes, axis=1) == np.argmax(true_classes_one_hot, axis=1))
print("Test Accuracy (Threshold):", accuracy)


55/55 [==============================] - 0s 2ms/step
Test Accuracy (Threshold): 0.9021058622652248


In [25]:
# model.save('model.h5')

c:\Users\prit4\OneDrive\Desktop\stuff\active_Github_repos\Mini_project\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
